In [1]:
import numpy as np
import pandas as pd
import yaml
import cv2
import os
import glob
import xml.etree.ElementTree as ET
import copy

In [2]:
xml_folder = '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/annotations/images-and-xmls/from-tracking/annotations-raw' 
xml_file_list = glob.glob(xml_folder + '/*', recursive=True)
print(len(xml_file_list), 'xml files found.')

272 xml files found.


In [3]:
pd.options.display.max_columns = 100


output_folder = xml_folder + '-mod/'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
sucess_count = 0
  
for xml_file in xml_file_list[:]:
    
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()

        new_xml = ET.Element('annotation')
        folder = ET.SubElement(new_xml, 'folder')
        folder.text = root.find('folder').text
        filename = ET.SubElement(new_xml, 'filename')
        filename.text = root.find('filename').text
        image_size = ET.SubElement(new_xml, 'size')
        size_old = root.find('size')
        image_width = ET.SubElement(image_size, 'width')
        image_width.text = size_old.find('width').text
        old_width = int(size_old.find('width').text)
        image_height = ET.SubElement(image_size, 'height')
        image_height.text = size_old.find('height').text
        old_height = int(size_old.find('height').text)
        image_depth = ET.SubElement(image_size, 'depth')
        if image_depth:
            image_depth.text = size_old.find('depth').text

        segmented = ET.SubElement(new_xml, 'segmented')
        segmented.text = '0'
        if root.find('object') == None:
            print('The following file had no object classes listed:')
            print(xml_file)
            continue
#         print(old_width, old_height)
        for animal in root.findall('object'):
            animal_object = ET.SubElement(new_xml, 'object')
            #name
            name = animal.find('name').text
            name = name.split('-')
            animal_name = name[0]
    #         animal_age = name[1]
            #deal with different names for same animal type
            if animal_name in {'PZ', 'plains_zebra','grevys_zebra', 'GZ', 'zebra'}:
                animal_name = 'zebra'
            elif animal_name in {'GG', 'TG', 'imp', 'gazelle'}:
                animal_name = 'gazelle'
            elif animal_name in {'wbuck'}:
                animal_name = 'wbuck'
            elif animal_name in {'giraffe', 'eland', 'warthog', 'other'}:
                animal_name = 'other'
            else:
                print('here')
                animal_name = 'other'
            name = ET.SubElement(animal_object, 'name')
            name.text = animal_name
            #pose
            pose = ET.SubElement(animal_object, 'pose')
            pose.text = 'Unspecified'
            #truncated
            truncated = ET.SubElement(animal_object, 'truncated')
            truncated.text = '0'
            #difficult
            difficult = ET.SubElement(animal_object, 'difficult')
            difficult.text = '0'

            bndbox_old = animal.find('bndbox')
            
            

            #bndbox
            bndbox = ET.SubElement(animal_object, 'bndbox')
            xmin = ET.SubElement(bndbox, 'xmin')
            old_xmin = int(bndbox_old.find('xmin').text)
            if old_xmin < 0:
                old_xmin = 0
            xmin.text = str(old_xmin)
            ymin = ET.SubElement(bndbox, 'ymin')
            old_ymin = int(bndbox_old.find('ymin').text)
            if old_ymin < 0:
                old_ymin = 0
            old_ymax = int(bndbox_old.find('ymax').text)
            if old_ymax >= old_height:
                print('here')
                old_ymax = old_height - 1
            if old_ymin >= old_ymax:
                temp = copy.copy(old_ymin)
                old_ymin = copy.copy(old_ymax)
                old_ymax = temp
            ymin.text = str(old_ymin)
            xmax = ET.SubElement(bndbox, 'xmax')
            old_xmax = int(bndbox_old.find('xmax').text)
            if old_xmax >= old_width:
                old_xmax = old_width - 1
            xmax.text = str(old_xmax)
            ymax = ET.SubElement(bndbox, 'ymax')
#             old_ymax = int(bndbox_old.find('ymax').text)
#             if old_ymax >= old_height:
#                 print('here')
#                 old_ymax = old_height - 1
            ymax.text = str(old_ymax)

        file_name, _ = os.path.splitext(root.find('filename').text) 
        outfile = output_folder +  file_name + str('.xml')

        tree = ET.ElementTree(new_xml)
        tree.write(outfile)
        sucess_count += 1
        
    except Exception as e:
        print('file in unexpected format:')
        print(e)
        print(xml_file)
print(sucess_count , 'files procesed')

here
272 files procesed


In [6]:
x = 'plains'
print(x in {'plains'})

True


eya
